In [3]:
#default_exp qdrant

In [26]:
#export
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from tqdm import tqdm

from qdrant_client import QdrantClient
collection_name = "food_prompted"

docker run -p 6333:6333 \
    -v $(pwd)/qdrant_storage:/qdrant/storage \
    --name qdrant \
    qdrant/qdrant
docker update --restart unless-stopped qdrant

In [27]:
# !pip install --upgrade qdrant_client

In [28]:
#export
client = QdrantClient(host='localhost', port=6333)
dim = 768 #+ onehot.n_dim()
client.get_collections = lambda: [c['name'] for c in client.http.collections_api.get_collections().dict()['result']['collections']]
client.collection_len = lambda name: client.http.collections_api.get_collection(name).dict()['result']['vectors_count']


In [29]:
client.get_collections()

['glovo', 'fundation_food', 'food']

In [40]:
# client.recreate_collection(collection_name,dim,qdrant_client.http.models.Distance.DOT)

In [41]:
client.get_collections()

['food_prompted', 'glovo', 'fundation_food', 'food']

In [47]:
collection_name

'food_prompted'

In [45]:
client.collection_len(collection_name)

7081

In [46]:
!nbdev_build_lib

Converted 00_clipmodel.ipynb.
Converted 00_custom_pandas.ipynb.
Converted 00_paths.ipynb.
Converted 00_psql.ipynb.
Converted 00_qdrant.ipynb.
Converted 00_tools.ipynb.
Converted 01_multiple_foods.ipynb.
Converted 01_multiple_foods_segmantation.ipynb.
Converted 01_search.ipynb.
Converted 0_template copy 2.ipynb.
Converted 0_template.ipynb.
Converted Untitled-1.ipynb.
Converted foodd dataset.ipynb.
Converted foods_prompted_tosql.ipynb.
Converted inference.ipynb.
Converted multiple_3005.ipynb.
No export destination, ignored:
#export
def search_image(url=None,head = 1):
    image_clip = requests.post(f'https://guru.skynet.center/image2vector/?url={url}').json()
    results = client.search(collection_name=collection_name,query_vector=image_clip,top=head)
    image_clip = torch.Tensor(image_clip)
    df = foods.loc[[r.id for r in results]].copy()
    df['score'] = [r.score for r in results]
    df = df.sort_values('score',ascending=False)

    return image_clip,df.reset_index()

series2tenso